In [1]:
import pandas as pd
import numpy as np
import plotnine as p9
import pyfpgrowth as fp

import matplotlib.pyplot as plt
%matplotlib inline

import datetime
import time

In [2]:
# Reading JSON data into python as pandas dataframe.

data = pd.read_json('C:/Semester 2/USML/Project_Final/AMAZON_FASHION.json',lines=True)
meta_data = pd.read_json('C:/Semester 2/USML/Project_Final/meta_AMAZON_FASHION.json',lines=True)
data

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,NaN,NaN
1,2,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3.0,NaN,NaN
2,4,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,NaN,NaN
3,2,True,"08 24, 2014",A2UH2QQ275NV45,7106116521,Jodi Stoner,too tiny an opening,Two Stars,1408838400,NaN,NaN,NaN
4,3,False,"07 27, 2014",A89F3LQADZBS5,7106116521,Alexander D.,Okay,Three Stars,1406419200,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
883631,5,True,"02 21, 2017",A1ZSB2Q144UTEY,B01HJHTH5U,Amazon Customer,I absolutely love this dress!! It's sexy and ...,I absolutely love this dress,1487635200,NaN,NaN,NaN
883632,5,True,"11 25, 2016",A2CCDV0J5VB6F2,B01HJHTH5U,Amazon Customer,I'm 5'6 175lbs. I'm on the tall side. I wear a...,I wear a large and ordered a large and it stil...,1480032000,2.0,NaN,NaN
883633,3,True,"11 10, 2016",A3O90PACS7B61K,B01HJHTH5U,Fabfifty,Too big in the chest area!,Three Stars,1478736000,NaN,NaN,NaN
883634,3,True,"11 10, 2016",A2HO94I89U3LNH,B01HJHF97K,Mgomez,"Too clear in the back, needs lining",Three Stars,1478736000,NaN,NaN,NaN


In [3]:
# Converting unix time to Date format and storing it in reviewTime Column.

data['reviewTime']=data.apply(lambda row: list(time.gmtime(row.unixReviewTime))[:-6], axis=1)
data = data.sort_values('unixReviewTime')
data['year']=data.apply(lambda row: row.reviewTime[0], axis=1)

# We will Make 2 different User item matrix.#

## 1. With items with overall rating of 4 and above.

## 2. With all the items, including items with bad ratings.

## Method to create user itemsets.

In [4]:
def make_useritem_matrix(byreviewer):
    reviewers = list(byreviewer.reviewerID.unique())
    itemratedf = pd.DataFrame(data={'reviewerID':reviewers})
    for index, row in byreviewer.iterrows():
        itemratedf.loc[itemratedf['reviewerID'] == row['reviewerID'], row['asin']] = row['overall']
    return itemratedf

# 1. Creating data with items with overall rating of 4 and above.

In [5]:
# Now we will prepare data to make recommendations only based on reviews which have 4 or more ratings.
# We will delete all the rows with overall ratings less than 4.
# Then we remove all customers who have made less than 9 transactions.
# It is hard to learn behaviour of a customer with just one or two transactions hence removing the customers with less
# transactions will help us learn better.

data.drop(data[ data['overall'] < 4 ].index , inplace=True)

byreviewer = data.groupby('reviewerID')
byreviewer = byreviewer.filter(lambda x: len(x) > 9)
byreviewer

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image,year
400100,5,True,"[2007, 12, 18]",AWBGHDHH7E51F,B000HX58GG,Trend,The picture does not show just how beautiful t...,Beautiful,1197936000,NaN,NaN,NaN,2007
400070,5,True,"[2008, 1, 1]",AWBGHDHH7E51F,B000HX3BQ0,Trend,This pendant has a beautiful stone. It is a v...,Beautiful Blue Topaz,1199145600,NaN,NaN,NaN,2008
400295,5,True,"[2008, 1, 3]",AWBGHDHH7E51F,B000IBVRGC,Trend,"The way the earrings are designed, makes the s...",Little Drops of Sparkle,1199318400,NaN,NaN,NaN,2008
400228,5,True,"[2008, 1, 9]",AWBGHDHH7E51F,B000IBUM1I,Trend,These are pretty earrings. The stones are ver...,Attractive and Well Made,1199836800,NaN,NaN,NaN,2008
400309,5,True,"[2008, 1, 16]",AWBGHDHH7E51F,B000IBT16K,Trend,The color of these stones are as rich and sat...,"A ""Pop"" of color",1200441600,NaN,NaN,NaN,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...
592473,5,True,"[2018, 4, 26]",A2QHRRGSHKUZ6I,B00RT3Y4RQ,Amazon Customer,Love it! So warm and cozy!,Super cute!,1524700800,NaN,"{'Size:': ' Large (14)', 'Color:': ' Light Gray'}",NaN,2018
349997,5,True,"[2018, 5, 5]",AKMEY1BSHSDG7,B01A7D4QH2,J. Arena,Do not hesitate to purchase these compression ...,F A B U L O U S !!!,1525478400,3.0,"{'Size:': ' 4-10', 'Color:': ' White'}",NaN,2018
577951,5,True,"[2018, 6, 9]",A2T2ZJ8NQ7PP5J,B00PJIO5PO,Ruth U.,Beautiful. Well made. Wish there were earrin...,Five Stars,1528502400,NaN,NaN,NaN,2018
709128,4,True,"[2018, 8, 26]",A3G5KDMFNRUXHB,B015HAX52C,michelle williams brewster,Excellent fit,Four Stars,1535241600,NaN,"{'Size:': ' 40G', 'Color:': ' In the Navy Scro...",NaN,2018


## Saving data and user itemset with items with overall rating of 4 and above.

In [6]:
itemratedf = make_useritem_matrix(byreviewer)
itemratedf.to_csv(r'C:/Semester 2/USML/Project_Final/AMAZON_FASHION_posonly_user_itemset.csv', index = False)
byreviewer.to_csv(r'C:/Semester 2/USML/Project_Final/AMAZON_FASHION_posonly_user_clean.csv', index = False)

## 2. Creating data with all the items, including items with bad ratings.

In [7]:
data = pd.read_json('C:/Semester 2/USML/Project_Final/AMAZON_FASHION.json',lines=True)

# We remove all customers who have made less than 9 transactions.
# It is hard to learn behaviour of a customer with just one or two transactions hence removing the customers with less
# transactions will help us learn better.

byreviewer = data.groupby('reviewerID')
byreviewer = byreviewer.filter(lambda x: len(x) > 9)

 ## Saving data and user itemset with items with all the items, including items with bad ratings.

In [8]:
itemratedf = make_useritem_matrix(byreviewer)
itemratedf.to_csv(r'C:/Semester 2/USML/Project_Final/AMAZON_FASHION_user_itemset.csv', index = False)
byreviewer.to_csv(r'C:/Semester 2/USML/Project_Final/AMAZON_FASHION_clean.csv', index = False)

# We will now do EDA on the cleaned Data in EDA.ipynb